<h1 style="text-align: center;">Gradient Descent Without Backpropagation</h1>
<h4 style="text-align: center;">Author: Ignacio &Aacute;vila Reyes</h4>

## Introduction

Using backpropagation to compute gradients of functios in order to design machine learning has been the order of the day for a long time.

Here we present an alternative method which is called **Forward Gradient** and its main advantage is computing the gradient during the forward step. Roughly speaking, this is an unbiased estimate of the gradient that permits us to entirely remove the backward step during the training of a neural network.

Let's explain briefly each one of the methods:

## Forward Mode

Given a function $f:\mathbb{R}^n\longrightarrow\mathbb{R}^m$ and the values $\theta\in\mathbb{R}^n$, $v\in\mathbb{R}^n$. *Forward Mode* computes $f(\theta)$ and the jacobian vector product $J_f(\theta)\cdot v$ where $v$ is a vector of perturbations. All of this computed in just the **Forward Step**.

<div align="center"><img alt="Forward Step Scheme" src="./src/images/fwdStep.png"></div>

Firstly, let's get in touch with the method:

In [18]:
import torch
import tensorflow
import functorch as fc

In [19]:
# Let's try to use CUDA
DEVICE = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [32]:
# We define an input
input = torch.tensor([[5., 10.]]).to(DEVICE)

# Our random vector
v = torch.randn_like(input)
print(v)

# A function
def f(x):
    return x**2

tensor([[ 0.6185, -1.1474]])


In [33]:
# funtorch.jvp(f, input, vector) returns f(input) and the directional gradient of
# "f" in "input" with direction "vector"
value, grad = fc.jvp(f, (input,), (v.to(DEVICE),))
print("Results:", value)
print("Gradient:",grad)


Results: tensor([[ 25., 100.]])
Gradient: tensor([[  6.1849, -22.9488]])


## Reverse Mode

Given a function $f:\mathbb{R}^n\longrightarrow\mathbb{R}^m$ and the values $\theta\in\mathbb{R}^n$, $v\in\mathbb{R}^n$. *Reverse Mode* computes $f(\theta)$ and the vector-jacobian product $v^T\cdot J_f(\theta)$ where $v$ is a vector of adjoints.

<div align="center"><img alt="Backward Step Scheme" src="./src/images/bckStep.png"></div>

We have already got in touch with this during lab sessions, but let's make some basic calculations:

In [93]:
loss = torch.nn.MSELoss()

# Inputs and expected predictions
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)

print("Input:", input)
print("Target:", target)

# We define a model with a simple activation layer using the relu
# function
pred = input.relu()
print("Prediction:", pred)
out = loss(input , pred)
out.backward()

# Gradient
print("Gradient:", input.grad)

Input: tensor([[ 1.0963, -1.5237,  0.2426, -0.3973, -1.2430],
        [ 0.9085, -0.5564, -2.6606,  0.9741, -0.2563],
        [ 0.1526, -0.3247, -0.7983, -0.0370, -2.5105]], requires_grad=True)
Target: tensor([[ 0.9612, -1.4112, -0.5299, -0.3602, -1.5632],
        [-1.4272,  0.2704,  0.4153,  1.3643,  0.6472],
        [-0.6394, -0.3596,  0.0329, -0.7257, -0.4160]])
Prediction: tensor([[1.0963, 0.0000, 0.2426, 0.0000, 0.0000],
        [0.9085, 0.0000, 0.0000, 0.9741, 0.0000],
        [0.1526, 0.0000, 0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)
Gradient: tensor([[ 0.0000, -0.2032,  0.0000, -0.0530, -0.1657],
        [ 0.0000, -0.0742, -0.3547,  0.0000, -0.0342],
        [ 0.0000, -0.0433, -0.1064, -0.0049, -0.3347]])


# Method

## Forward Gradients

**Definition.** Given a function $f:\mathbb{R}^n\longrightarrow\mathbb{R}$ we define the "forward gradient" $g:\mathbb{R}^n\longrightarrow\mathbb{R}^n$ as $g(\theta)=(\nabla f(\theta)\cdot v)\hspace{0.1cm}v)$ where $\theta\in\mathbb{R}^n$ is the point at which we are evaluating the gradient and $v\in\mathbb{R}^n$ is a perturbation vector taken as a multivariate random variable $v\sim p(v)$ such that $v_i$ components has *zero mean* and *unit variance*. So that $\nabla f(\theta)\cdot v$ is the directional derivative of $f$ at point $\theta$ in direction $v$.

So each time we evaluate the **forward gradient**, we simply do the following:
- Sample random perturbation vector $v\sim p(v)$
- Evaluate $f(\theta)$ and $\nabla f(\theta)\cdot v$ simoultaneously in the same single forward step without having to compute $\nabla f$ at all in the process.
- Multiply the scalar directional derivative $\nabla f(\theta)\cdot v$ and obtain $g(\theta)$, the forward gradient.

<div align="center"><img alt="Forward Gradient Graph" src="./src/images/graph.png"></div>


# Backpropagation

In [ ]:
if torch.cuda.is_available():
    print("Using ", torch.cuda.get_device_name())
else:
    print("Using CPU")
    
!python .\src\global_optimization_backprop.py

# Forward Propagation

In [ ]:
if torch.cuda.is_available():
    print("Using ", torch.cuda.get_device_name())
else:
    print("Using CPU")
    
!python .\src\global_optimization_fwdgrad.py